# American Sign Language (ASL) Final Project W251 YOLOv5 Custom Dataset
---
 
## Set up on Nvidia Jetson NX

- Using Docker Image Inference which uses the base image nvcr.io/nvidia/l4t-pytorch:r32.6.1-pth1.9-py3 from the Dockerfile.HandDetector
- Building Docker image

```bash
sudo docker build -t mediapipe -f Dockerfile.HandDetector .
```
- Run the container and mounting data in the /app/ folder

```bash
docker run --privileged --runtime nvidia  --device /dev/video0  --network host --rm -v /data/w251:/app/w251 -e DISPLAY=$DISPLAY -v /tmp:/tmp -ti mediapipe
```
- Run the following script after spinning up the Docker Container to remove the conficted Pillow version

```bash
bash dependencies_setup.sh
```
- Run the following line before spinning up Jupyter to update Jupyter config

```bash
export LD_PRELOAD=/usr/lib/aarch64-linux-gnu/libgomp.so.1
```
- Spin up jupyter lab

```bash
jupyter lab --no-browser --ip=0.0.0.0 --allow-root
```

#### 1. Download Yolov5 Repository & Data

In [ ]:
!pwd

In [ ]:
# !git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5

In [ ]:
import torch

from IPython.display import Image, clear_output  # to display images

clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

#### 2. Make sure to download the dataset from Roboflow as YOLO v5 PyTorch

In [ ]:
!mkdir ./data/asl7

In [ ]:
%cd ./data/asl7

In [ ]:
!curl -L "https://app.roboflow.com/ds/KnljOtVDPM?key=37TDe9XWw3" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

In [ ]:
%cat data.yaml

In [ ]:
import yaml
with open("data.yaml", 'r') as stream:
    num_classes = str(yaml.safe_load(stream)['nc'])

In [ ]:
# cat the model used
#%cat /content/yolov5/models/yolov5s.yaml

In [ ]:
#customize iPython writefile so we can write variables
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [ ]:
%cd ../..

In [ ]:
print(num_classes)

In [ ]:
%%writetemplate models/custom_yolov5s.yaml

# parameters
nc: {num_classes}  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple

# anchors
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Focus, [64, 3]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, BottleneckCSP, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 9, BottleneckCSP, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, BottleneckCSP, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 1, SPP, [1024, [5, 9, 13]]],
   [-1, 3, BottleneckCSP, [1024, False]],  # 9
  ]

# YOLOv5 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, BottleneckCSP, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, BottleneckCSP, [256, False]],  # 17 (P3/8-small)

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, BottleneckCSP, [512, False]],  # 20 (P4/16-medium)

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, BottleneckCSP, [1024, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]

In [ ]:
!ls

## Inference with Jetson NX - 32FP

In [ ]:
!pwd

In [ ]:
torch.cuda.empty_cache()
torch.cuda.is_available()

In [ ]:
# Validation with ASL model
!python3 val.py --weights ./weights/final_1_best_yolov5_300_epochs.pt --data ./data/asl7/data.yaml

In [ ]:
# Running Inference with a test picture - PF32
!python3 detect.py --source test.jpg --weights ./weights/final_1_best_yolov5_300_epochs.pt --conf 0.4 --data ./data/asl7/data.yaml

In [ ]:
# Running Inference with live camera - PF32
!python3 detect.py --source 0 --weights ./weights/final_1_best_yolov5_300_epochs.pt --conf 0.4 --data ./data/asl7/data.yaml

## Inference with Jetson NX - Tensort RT

In [ ]:
# Export to tensortrt models
!python3 export.py --weights ./weights/final_1_best_yolov5_300_epochs.pt  --include torchscript engine --device 0

In [ ]:
# Tensort RT model - evaluation
!python3 val.py --weights ./weights/final_1_best_yolov5_300_epochs.engine --data ./data/asl7/data.yaml

In [ ]:
# Tensort RT model - live camera inference
!python3 detect.py --source 0 --weights ./weights/final_6_best_yolov5_300_epochs.engine --conf 0.4 --data ./data/asl7/data.yaml

## Inference with Jetson NX - Torchscript

In [ ]:
# Torchscript model - evaluation
!python3 val.py --weights ./weights/final_1_best_yolov5_300_epochs.torchscript --data ./data/asl7/data.yaml

In [ ]:
# Torchscript model - live camera inference
!python3 detect.py --source 0 --weights ./weights/final_1_best_yolov5_300_epochs.torchscript --conf 0.4 --data ./data/asl7/data.yaml

## Inference with Jetson NX - ONNX

In [ ]:
# ONNX - evaluation
!python3 val.py --weights ./weights/final_1_best_yolov5_300_epochs.onnx --data ./data/asl7/data.yaml

In [ ]:
#ONNX model - live camera inference
!python3 detect.py --source 0 --weights ./weights/final_1_best_yolov5_300_epochs.onnx --conf 0.4 --data ./data/asl7/data.yaml